<a href="https://colab.research.google.com/github/jonrtaylor/twitch/blob/master/example_model_nomi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
####THIS NOTEBOOK IS DEPRECATED AND SHOULD NOT BE USED####

import pandas as pd
import numpy as np

df_train = pd.read_csv('https://cloudflare-ipfs.com/ipfs/QmX1cDy6iqKGy8B4wSx8PkwxDTgPmWuZZBTM7tL1WNizv8/numerai_training_validation_target_nomi.csv.tar.gz')

In [ ]:
df_train = df_train.iloc[:-1].copy()

df_train['era'] = df_train.loc[:, 'era'].str[3:].astype(int)
df_val = df_train[df_train.era > 120].copy()

In [ ]:
df_training = df_train[df_train.era < 121].copy()
del df_train

In [ ]:
from xgboost import XGBRegressor

model = XGBRegressor(max_depth = 5, learning_rate = 0.01, n_estimators = 2000, n_jobs = -1, colsample_bytree = 0.1)#, tree_method = 'gpu_hist')
#slight change to save resources; still works!
#y = df_training.target_nomi.values

features = [c for c in df_training if c.startswith("feature")]

#slight change to save resources; still works!
#X = df_training[features].values

model.fit(df_training[features].values, df_training.target_nomi.values)

[17:14:55] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.1, gamma=0,
             importance_type='gain', learning_rate=0.01, max_delta_step=0,
             max_depth=5, min_child_weight=1, missing=None, n_estimators=2000,
             n_jobs=-1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [ ]:
def correlation_score(y_true, y_pred):
    return np.corrcoef(y_true, y_pred)[0,1]
#y_val = df_val.target_nomi.values
#X_val = df_val[features].values
preds = model.predict(df_val[features].values)

In [ ]:
nomi_score = correlation_score(df_val.target_nomi.values, preds)
print(nomi_score)

0.028289112027785463


In [ ]:
#y_valk = df_val.target_kazutsugi.values

In [ ]:
kaz_score = correlation_score(df_val.target_kazutsugi.values, preds)
print(kaz_score)
del df_val
del df_training

0.030594348716166283


In [ ]:
tournament_data = pd.read_csv("https://numerai-public-datasets.s3-us-west-2.amazonaws.com/latest_numerai_tournament_data.csv.xz")
#X_tournament = tournament_data[features].values
predictions = model.predict(tournament_data[features].values)
predictions_df = tournament_data["id"].to_frame()
predictions_df["prediction_kazutsugi"] = predictions
predictions_df.to_csv('example_predictions_nomi_twitch.csv', index=False)

In [ ]:
from google.colab import files
files.download("example_predictions_nomi_twitch.csv")